In [30]:
import numpy as np
from scipy.linalg import eig
from numpy.linalg import cond

def print_results(eigenvalues, condition_number):
    print("Eigenvalues:")
    print(eigenvalues)
    print("="*80)
    print("Condition Number: {:.4e}".format(condition_number))

def compute_eigenvalues_and_condition_number(n, R):
    # Generate randomly weights and biases and split into two
    b = R * np.random.rand(2 * n)
    w = R * np.random.rand(2 * n)
    b1, b2 = b[:n], b[n:]
    w1, w2 = w[:n], w[n:]
    
    L = np.pi  # size of the interval
    delta = L / 8  # overlap
    
    # Initialize matrices
    M1 = np.zeros((n, n))
    M1t = np.zeros((n, n))
    M2 = np.zeros((n, n))
    M2t = np.zeros((n, n))
    M12 = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            # Compute for M1 and M1t
            wij = w1[i] - w1[j]
            vij = w1[i] + w1[j]
            bij = b1[i] - b1[j]
            cij = b1[i] + b1[j]
            if wij != 0:
                M1[i, j] = -(np.sin(0.5 * vij * L + vij * delta + cij) * wij -
                             np.sin(0.5 * wij * L + wij * delta + bij) * vij -
                             np.sin(cij) * wij + np.sin(bij) * vij) / (vij * wij)
                M1t[i, j] = -1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (2 * np.sin(0.5 * wij * L - wij * delta + bij) * delta**2 * vij**3 * wij**2 -
                         2 * np.sin(0.5 * vij * L - vij * delta + cij) * delta**2 * vij**2 * wij**3 -
                         2 * np.cos(0.5 * wij * L - wij * delta + bij) * delta * vij**3 * wij +
                         2 * np.cos(0.5 * vij * L - vij * delta + cij) * delta * vij * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) -
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) +
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M1[i, j] = (np.cos(bij) * w1[i] * L + 2 * np.cos(bij) * w1[i] * delta +
                            np.sin(cij) - np.sin(L * w1[i] + 2 * delta * w1[i] + cij)) / (2 * w1[i])
                M1t[i, j] = 1 / (12 * delta**2 * w1[i]**3) * (
                    32 * np.cos(bij) * delta**3 * w1[i]**3 +
                    24 * delta**2 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij) * w1[i]**2 -
                    12 * np.cos(L * w1[i] - 2 * delta * w1[i] + cij) * delta * w1[i] +
                    3 * np.sin(L * w1[i] + 2 * delta * w1[i] + cij) -
                    3 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij))
            
            # Compute for M2 and M2t
            wij = w2[i] - w2[j]
            vij = w2[i] + w2[j]
            bij = b2[i] - b2[j]
            cij = b2[i] + b2[j]
            if wij != 0:
                M2[i, j] = -(np.sin(0.5 * vij * L + vij * delta + cij) * wij -
                             np.sin(0.5 * wij * L + wij * delta + bij) * vij -
                             np.sin(cij) * wij + np.sin(bij) * vij) / (vij * wij)
                M2t[i, j] = 1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (2 * np.sin(0.5 * wij * L + wij * delta + bij) * delta**2 * vij**3 * wij**2 -
                         2 * np.sin(0.5 * vij * L + vij * delta + cij) * delta**2 * vij**2 * wij**3 +
                         2 * np.cos(0.5 * wij * L + wij * delta + bij) * delta * vij**3 * wij -
                         2 * np.cos(0.5 * vij * L + vij * delta + cij) * delta * vij * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) +
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) -
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M2[i, j] = (np.cos(bij) * w2[i] * L + 2 * np.cos(bij) * w2[i] * delta +
                            np.sin(cij) - np.sin(L * w2[i] + 2 * delta * w2[i] + cij)) / (2 * w2[i])
                M2t[i, j] = -1 / (12 * delta**2 * w2[i]**3) * (
                    -32 * np.cos(bij) * delta**3 * w2[i]**3 +
                    24 * delta**2 * np.sin(L * w2[i] + 2 * delta * w2[i] + cij) * w2[i]**2 +
                    12 * np.cos(L * w2[i] + 2 * delta * w2[i] + cij) * delta * w2[i] -
                    3 * np.sin(L * w2[i] + 2 * delta * w2[i] + cij) +
                    3 * np.sin(L * w2[i] - 2 * delta * w2[i] + cij))
            
            # Compute for M12
            wij = w1[i] - w2[j]
            vij = w1[i] + w2[j]
            bij = b1[i] - b2[j]
            cij = b1[i] + b2[j]
            if wij != 0:
                M12[i, j] = -1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (np.cos(0.5 * wij * L + wij * delta + bij) * delta * vij**3 * wij -
                         np.cos(0.5 * vij * L + vij * delta + cij) * delta * vij * wij**3 +
                         np.cos(0.5 * wij * L - wij * delta + bij) * delta * vij**3 * wij -
                         np.cos(0.5 * vij * L - vij * delta + cij) * delta * vij * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) +
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) -
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M12[i, j] = 1 / (12 * delta**2 * w1[i]**3) * (
                    16 * np.cos(bij) * delta**3 * w1[i]**3 +
                    6 * np.cos(L * w1[i] + 2 * delta * w1[i] + cij) * delta * w1[i] +
                    6 * np.cos(L * w1[i] - 2 * delta * w1[i] + cij) * delta * w1[i] -
                    3 * np.sin(L * w1[i] + 2 * delta * w1[i] + cij) +
                    3 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij))

    # Form the full matrix
    M_full = np.block([[M1 + M1t, M12], [M12.T, M2 + M2t]])
    
    # Compute eigenvalues
    eigenvalues = eig(M_full)[0]
    
    # Compute condition number
    condition_number = cond(M_full)
    
    print_results(eigenvalues, condition_number)
    
    return eigenvalues, condition_number

In [31]:
n = 4
R = 1
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[1.57020537e+01+0.j 7.02183248e+00+0.j 1.19593283e-01+0.j
 5.51019990e-02+0.j 1.12051445e-03+0.j 1.04200567e-04+0.j
 3.78144463e-09+0.j 1.21781707e-06+0.j]
Condition Number: 4.1524e+09


In [32]:
n = 4
R = 2
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[1.51935139e+01+0.j 8.98059889e+00+0.j 2.83000422e+00+0.j
 2.20154519e+00+0.j 4.14042927e-02+0.j 2.01333229e-02+0.j
 1.23874208e-05+0.j 4.27802056e-05+0.j]
Condition Number: 1.2265e+06


In [33]:
n = 4
R = 4
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[12.28223669+0.j  7.89848349+0.j  3.48930954+0.j  4.4315811 +0.j
  1.57181191+0.j  0.09364597+0.j  0.01704627+0.j  0.02516108+0.j]
Condition Number: 7.2052e+02


In [34]:
n = 4
R = 8
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[6.09601896+0.j 0.54438559+0.j 0.22379801+0.j 3.74647399+0.j
 1.73466476+0.j 2.98117901+0.j 2.6520627 +0.j 2.18486887+0.j]
Condition Number: 2.7239e+01


In [35]:
n = 4
R = 16
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[7.75251439+0.j 5.44372028+0.j 0.21695148+0.j 0.15795116+0.j
 4.24947659+0.j 2.12558577+0.j 2.35326411+0.j 2.4416427 +0.j]
Condition Number: 4.9082e+01


In [36]:
n = 4
R = 32
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[6.98804737+0.j 3.9747343 +0.j 3.80555758+0.j 3.22693616+0.j
 2.77309122+0.j 2.1395721 +0.j 2.26863304+0.j 2.51627315+0.j]
Condition Number: 3.2661e+00


In [37]:
n = 4
R = 64
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[0.41000069+0.j 5.69316853+0.j 5.44516124+0.j 1.20540439+0.j
 2.34402466+0.j 3.00998751+0.j 3.25009673+0.j 3.19277815+0.j]
Condition Number: 1.3886e+01


In [38]:
n = 4
R = 128
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[3.80429291+0.j 2.28430222+0.j 2.8164818 +0.j 2.90100227+0.j
 3.07534272+0.j 3.04236265+0.j 2.95873785+0.j 2.99550959+0.j]
Condition Number: 1.6654e+00


In [39]:
n = 4
R = 8
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[8.40460193+0.j 6.224773  +0.j 0.12313323+0.j 0.17962945+0.j
 1.06869016+0.j 3.78241023+0.j 3.55962586+0.j 3.05814846+0.j]
Condition Number: 6.8256e+01


In [44]:
n = 10
R = 128
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[6.38416303+0.j 5.44282547+0.j 0.11727376+0.j 4.28808689+0.j
 1.57731915+0.j 3.96262209+0.j 1.78033238+0.j 1.98073454+0.j
 2.07974379+0.j 3.72381952+0.j 3.60906129+0.j 2.26056716+0.j
 3.34748239+0.j 3.27859124+0.j 3.29757153+0.j 2.45115268+0.j
 2.89296455+0.j 2.71246993+0.j 2.67460149+0.j 2.69235775+0.j]
Condition Number: 5.4438e+01
